In [1]:
import geopandas as gpd
import pandas as pd
from google_geocoding import *
from feature_dev import *
from utils import *

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_rows', None)

# df = develop_features()

In [2]:
# Get station points
gdf = load_station_points()

# Extract lats and lons
gdf['lat'] = gdf.geometry.y
gdf['lon'] = gdf.geometry.x

# Make cartesian product of stations (all OD pairs)
df = pd.merge(gdf[['station_name','lat','lon']], gdf[['station_name','lat','lon']], how='cross', suffixes=('_o', '_d'))

# Remove diagonal
df = df[df.station_name_o != df.station_name_d]

In [6]:
len(df)

9312

In [8]:
dict = open_routing_results()

In [10]:
df = summarize_routes(dict)

In [12]:
df[[
    'google_driving_minutes',
    'google_transit_minutes',
    'google_bicycling_minutes',
    'google_walking_minutes',
]]

,google_driving_minutes,google_transit_minutes,google_bicycling_minutes,google_walking_minutes
1,20.05,36.20,47.57,141.25
2,26.68,32.15,52.35,158.78
3,25.37,40.25,68.00,218.35
4,55.47,91.53,235.27,840.38
5,33.87,45.75,91.72,293.57
